Decompose video into frames

In [5]:
from imutils import contours 
from skimage import measure  
import numpy as np          
import argparse
import imutils
import matplotlib.pyplot as plt
import cv2

vidcap = cv2.VideoCapture('VID_20220519_132403.mp4')

success, image = vidcap.read()
count = 1
while success:
  cv2.imwrite("pattern/image_%d.jpg" % count, image)    
  success, image = vidcap.read()
  count += 1

print("Saved", count, "images")

Saved 346 images


In [6]:
def plot_imshow_in_array(matrices_to_show:list , max_image_in_row:int = 4 , plots_kwargs_list:list = []):
    """
        Plots the image list in an array of row * col , where max(col) = max_image_in_row, and the 
        number of rows is calculated accordingly. 
        @param matrices_to_show a list of matrices to be plotted using plt.imshow
        @param max_image_in_row maximum number of plots in one row
        @param plots_kwargs_list list of dictionaries of parameters of each plot
        
    """    
    ## total number of plots
    plot_count = len(matrices_to_show)
    
    assert len(plots_kwargs_list) == 0 or len(plots_kwargs_list) == len(matrices_to_show) , """ plots_kwargs_list should be either an empty 
                                                                                    list or should have the same number of members
                                                                                    as matrices_to_show """
    
    ## number of rows
    nrow = plot_count // max_image_in_row + (1 if (plot_count % max_image_in_row > 0) else 0)
    
    ## number of columns
    ncol = max_image_in_row if plot_count >= max_image_in_row else plot_count
    
    ## plotting using imshow
    fig , ax = plt.subplots(nrow , ncol , figsize = (12 , 6))
    for i in range(plot_count):
        
        ## select the subplot
        plt.subplot(nrow , ncol , i+1 )
        
        ## format the argument dictionary of the imshow function
        if(len(plots_kwargs_list) == 0):
            config_dict = {}
        else: 
            config_dict = plots_kwargs_list[i]
        
        ## pass the image as well as extra arguments to the imshow function
        plt.imshow(matrices_to_show[i] , **config_dict)
        
    return fig , ax 

In [12]:
pictures = []

for i in range(1, count, 10):

    image_address = "./pattern/image_" +str(i) + ".jpg"

    ## read the image
    image = cv2.imread(image_address)

    ## convert BGR to RGB, we won't use this in any of the processes, just to test image loading
    rgb_image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB) 

    ## convert BGR to greyscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    ## blur the image with a gussian filter to remove any noise, and also to soften the image
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)

    thresh = cv2.threshold(blurred, 50 , 255 , cv2.THRESH_BINARY)[1]

    # ## plot images in a nice array. We have to define the colormap for imshow to get a black and white image, as the default 
    # ## color map shows colors between yellow (pixel = 255) and black (pixel = 0)
    # fig , ax = plot_imshow_in_array([rgb_image , gray , blurred, thresh] , \
    #                                 max_image_in_row = 4 , \
    #                                 plots_kwargs_list = [{} , {'cmap':'gray', 'vmin':0, 'vmax':255} , {'cmap':'gray', 'vmin':0, 'vmax':255}, {}] )
    (rows, columns) = thresh.shape[:2]

    best_row_index = 0
    best_row_count = 0

    for row in range(0, rows):
        cur_count = 0
        for column in range(0, columns):
            # count 255s in row
            if (thresh[row, column] == 255):
                cur_count += 1
        if cur_count > best_row_count:
            best_row_index = row
            best_row_count = cur_count
        if best_row_count > 0 and cur_count == 0:
            break

    # thresh[best_row_index,:] = 255

    regions = []

    last = -1
    cnt = 0
    for j in range(0, columns):
        if last != -1 and thresh[best_row_index, j] != last:
            last = thresh[best_row_index, j]
            regions.append(cnt)
            cnt = 0
        last = thresh[best_row_index, j]
        cnt += 1

    regions.append(cnt)

    pictures.append(regions)
    print("Finished picture " + str(i))

print(pictures)


Finished picture 1
Finished picture 11
Finished picture 21
Finished picture 31
Finished picture 41
Finished picture 51
Finished picture 61
Finished picture 71
Finished picture 81
Finished picture 91
Finished picture 101
Finished picture 111
Finished picture 121
Finished picture 131
Finished picture 141
Finished picture 151
Finished picture 161
Finished picture 171
Finished picture 181
Finished picture 191
Finished picture 201
Finished picture 211
Finished picture 221
Finished picture 231
Finished picture 241
Finished picture 251
Finished picture 261
Finished picture 271
Finished picture 281
Finished picture 291
Finished picture 301
Finished picture 311
Finished picture 321
Finished picture 331
Finished picture 341
[[225, 84, 92, 114, 95, 106, 364], [221, 11, 100, 111, 94, 108, 103, 72, 260], [228, 75, 93, 112, 96, 106, 370], [230, 87, 43, 161, 46, 155, 56, 30, 272], [232, 150, 43, 160, 50, 149, 296], [227, 5, 57, 158, 45, 156, 53, 106, 273], [254, 54, 42, 61, 41, 61, 43, 57, 48, 53, 53

In [31]:
features = [] #count blue, count yellow, width blue, width yellow, standard dev yellow?


for i in range(len(pictures)):
    
    circle = pictures[i][1:len(pictures[i])-1]
    # print(pictures[i])
    print(circle)
    feature = []
    if len(circle) % 2 ==0:
        feature.append(len(circle)/2)
        feature.append(len(circle)/2) #yellow
    else:
        feature.append((len(circle)-1)/2)
        feature.append((len(circle)+1)/2) #yellow
    
    yellow = 0
    blue = 0
    for j in range(len(circle)):
        if j % 2 == 0:
            yellow += circle[j]
        else:
            blue += circle[j]
    yellow /= feature[1]
    blue /= feature[0]
    feature.append(blue)
    feature.append(yellow)
    print(feature)


[84, 92, 114, 95, 106]
[2.0, 3.0, 93.5, 101.33333333333333]
[11, 100, 111, 94, 108, 103, 72]
[3.0, 4.0, 99.0, 75.5]
[75, 93, 112, 96, 106]
[2.0, 3.0, 94.5, 97.66666666666667]
[87, 43, 161, 46, 155, 56, 30]
[3.0, 4.0, 48.333333333333336, 108.25]
[150, 43, 160, 50, 149]
[2.0, 3.0, 46.5, 153.0]
[5, 57, 158, 45, 156, 53, 106]
[3.0, 4.0, 51.666666666666664, 106.25]
[54, 42, 61, 41, 61, 43, 57, 48, 53, 53, 43]
[5.0, 6.0, 45.4, 54.833333333333336]
[41, 47, 59, 41, 61, 43, 58, 47, 54, 51, 48]
[5.0, 6.0, 45.8, 53.5]
[55, 42, 61, 41, 61, 44, 56, 48, 53, 54, 30]
[5.0, 6.0, 45.8, 52.666666666666664]
[62, 141, 62, 149, 50]
[2.0, 3.0, 145.0, 58.0]
[34, 146, 63, 144, 57]
[2.0, 3.0, 145.0, 51.333333333333336]
[56, 142, 63, 147, 54]
[2.0, 3.0, 144.5, 57.666666666666664]
[102, 93, 111, 97, 104]
[2.0, 3.0, 95.0, 105.66666666666667]
[110, 92, 110, 101, 94]
[2.0, 3.0, 96.5, 104.66666666666667]
[46, 95, 113, 93, 109, 103, 41]
[3.0, 4.0, 97.0, 77.25]
[57, 44, 162, 44, 156, 55, 66]
[3.0, 4.0, 47.6666666666666